In [69]:
import import_ipynb
from Task_2 import hamming_distance as HD
from lib import bn256
import random
import numpy as np
import copy

In [70]:
v1, v2 = (np.random.choice([-1, 1], 16) for _ in range(2))
v1, v2

(array([ 1,  1, -1, -1, -1, -1, -1,  1, -1,  1, -1,  1, -1, -1, -1, -1]),
 array([-1, -1, -1, -1,  1, -1, -1,  1, -1, -1,  1,  1,  1,  1,  1,  1]))

In [71]:
G1 = bn256.curve_G
G2 = bn256.twist_G

In [72]:
v1points = [G2.scalar_mul(bn256.order + i) for i in v1]
v2points  = [G1.scalar_mul(bn256.order + i) for i in v2]

In [73]:
type(v1points[0]), type(v2points[0])

(lib.bn256.curve_twist, lib.bn256.curve_point)

In [74]:
def inverse_G1(curve):
    inv = copy.copy(curve)
    inv.y = inv.y.additive_inverse()
    return inv
def inverse_G2(twist):
    inv = copy.copy(twist)
    inv.negate()
    return inv


In [75]:
def CIPE_setup(G1, G2, n):
    s, t = ([random.randrange(2, bn256.order) for i in range(n)] for _ in range(2))
    u, v = ([random.randrange(2, bn256.order) for i in range(n+2)] for _ in range(2))
    h1, h2 = (random.randrange(2, bn256.order) for _ in range(2))
    H1, H2 = G1.scalar_mul(h1), G2.scalar_mul(h2)
    gen1_h = [G1.scalar_mul(s[i]).add(H1.scalar_mul(t[i])) for i in range(n)]
    gen2_h = [G2.scalar_mul(u[i]).add(H2.scalar_mul(v[i])) for i in range(n+2)]
    msk = (s, t, u, v, gen1_h, gen2_h, H1, H2)
    return msk

In [76]:
def Registration(G2, msk, v1points):
    s, t, gen2_h, H2 = msk[0], msk[1], msk[5], msk[7]
    n = len(s)
    r0 = random.randrange(2, bn256.order)
    reg_template = [None for _ in range(n+4)]
    reg_template[0] = G2.scalar_mul(r0)
    reg_template[1] = H2.scalar_mul(r0)
    for i in range(n+2):
        reg_template[i+2] = gen2_h[i].scalar_mul(r0)
    for i in range(n):
        reg_template[2] = reg_template[2].add(inverse_G2(v1points[i].scalar_mul(s[i])))
        reg_template[3] = reg_template[3].add(inverse_G2(v1points[i].scalar_mul(t[i])))
        reg_template[i+4] = reg_template[i+4].add(v1points[i])
    
    return reg_template

In [77]:
def Authentication(G1, msk, v2points):
    u, v, gen1_h, H1 = msk[2], msk[3], msk[4], msk[6]
    n = len(u)
    r0 = random.randrange(2, bn256.order)
    auth_template = [None for _ in range(n+2)]
    auth_template[0] = G1.scalar_mul(0)
    auth_template[1] = G1.scalar_mul(0)
    auth_template[2] = G1.scalar_mul(r0)
    auth_template[3] = H1.scalar_mul(r0)
    for i in range(n-2):
        auth_template[i+4] = gen1_h[i].scalar_mul(r0).add(v2points[i])
    for i in range(n):
        auth_template[0] = auth_template[0].add(inverse_G1(auth_template[i+2].scalar_mul(u[i])))
        auth_template[1] = auth_template[1].add(inverse_G1(auth_template[i+2].scalar_mul(v[i])))
    return auth_template

In [78]:
def Decryption(G1, G2, msk, v1points, v2points):
    reg_template = Registration(G2, msk, v1points)
    auth_template = Authentication(G1, msk, v2points)
    e = bn256.optimal_ate(reg_template[0], auth_template[0])
    n = len(auth_template)
    for i in range(1, n):
        e = e.mul(bn256.optimal_ate(reg_template[i], auth_template[i]))
    return reg_template, auth_template, e

In [79]:
def log_table(G1, G2, n):
    pair = bn256.optimal_ate(G2, G1)
    neg_pair = bn256.optimal_ate(G2, inverse_G1(G1))
    log_table = [pair.exp(i) for i in range(n, -1, -2)] + [neg_pair.exp(i) for i in range(2, n+1, 2)]
    return log_table

In [80]:
%%time
msk = CIPE_setup(G1, G2, len(v1points))
Decr = Decryption(G1, G2,  msk, v1points, v2points)
lgt = log_table(G1, G2, len(v1points))
e = Decr[2]
decr_dist = lgt.index(e)

Wall time: 21.1 s


In [81]:
HD(v1, v2), decr_dist


(9.0, 9)

In [82]:
HD(v1, v2) == decr_dist

True

ЗІЙШЛОСЬ

задача 4

In [83]:
from lib import bn256
import cv2

In [84]:
from IPython.display import Image
Image(filename='finger.jpg', width=300) 

In [85]:
au = cv2.imread("finger.jpg", 0)
au = au.flatten().astype('int')
print(au.shape)
n = au.shape[0]
reg = np.random.choice((-1,1), n)

(252,)


In [86]:
np.unique(au)

array([  0,   1,   2,   3,   4,   5,  10,  11,  12,  14,  17,  22,  24,
        28,  31,  32,  37,  41,  42,  46,  49,  61,  63,  64,  69,  71,
        72,  73,  74,  75,  78,  82,  84,  85,  88,  96, 100, 101, 103,
       105, 108, 113, 115, 116, 119, 127, 128, 131, 132, 135, 143, 147,
       148, 149, 163, 170, 173, 177, 179, 181, 192, 194, 196, 198, 203,
       212, 216, 218, 223, 227, 228, 231, 233, 234, 235, 236, 237, 238,
       241, 242, 244, 245, 246, 250, 251, 252, 253, 254, 255])

In [87]:
au[(au < 240)] = -1
au[(au >= 240)] = 1

In [88]:
au.shape, reg.shape

((252,), (252,))

In [89]:
G1 = bn256.curve_G
G2 = bn256.twist_G
pG2 = G2.scalar_mul(bn256.order + 1)
mG2 = G2.scalar_mul(bn256.order -1)
pG1 = G1.scalar_mul(bn256.order +1)
mG1 = G1.scalar_mul(bn256.order -1)
au_vec = [pG2 if i == 1 else mG2 for i in au]
reg_vec  = [pG1 if i == 1 else mG1 for i in reg]

In [90]:
%%time
msk = CIPE_setup(G1, G2, len(au_vec))

Wall time: 1min 27s


In [91]:
%%time
Decr = Decryption(G1, G2,  msk, au_vec, reg_vec)
e = Decr[2]

Wall time: 3min 10s


In [92]:
%%time
lgt = log_table(G1, G2, len(au_vec))
decr_dist = lgt.index(e)

Wall time: 2.49 s


In [93]:
HD(au, reg), decr_dist

(127.0, 127)

In [94]:
HD(au, reg) == decr_dist

True